In [1]:
import pandas as pd
import numpy as np

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [2]:
train_data = pd.read_parquet('../../data/initial_modelling/train_data_last_month.parquet')
test_data = pd.read_parquet('../../data/initial_modelling/test_data_last_month.parquet')

In [7]:
cat_cols = ['B_38', 'D_63', 'B_30', 'D_126']
train_data[cat_cols] = train_data[cat_cols].astype('category')
test_data[cat_cols] = test_data[cat_cols].astype('category')

In [8]:
train_data.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386034 entries, 0 to 386033
Data columns (total 95 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   customer_ID   386034 non-null  string        
 1   S_2           386034 non-null  datetime64[ns]
 2   D_39          386034 non-null  float32       
 3   B_1           386034 non-null  float32       
 4   B_2           386034 non-null  float32       
 5   R_1           386034 non-null  float32       
 6   D_41          386034 non-null  float32       
 7   B_3           386034 non-null  float32       
 8   B_4           386034 non-null  float32       
 9   D_45          386034 non-null  float32       
 10  B_5           386034 non-null  float32       
 11  R_2           386034 non-null  float32       
 12  D_47          386034 non-null  float32       
 13  B_6           386034 non-null  float32       
 14  B_7           386034 non-null  float32       
 15  D_51          386

In [4]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np

def one_hot_encode_categories(df, categorical_columns=None, drop_original=True, handle_unknown='error'):
    """
    One-hot encode categorical columns in a DataFrame.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        Input DataFrame containing categorical columns
    categorical_columns : list or None
        List of categorical column names to encode. If None, automatically detects categorical columns.
    drop_original : bool
        Whether to drop the original categorical columns
    handle_unknown : str
        Strategy for handling unknown categories in new data: 'error', 'ignore' or 'infrequent_if_exist'
    
    Returns:
    --------
    pandas.DataFrame
        DataFrame with one-hot encoded columns
    OneHotEncoder
        Fitted encoder for future transformations
    """
    # Make a copy to avoid modifying the original
    result_df = df.copy()
    
    # Automatically detect categorical columns if not specified
    if categorical_columns is None:
        categorical_columns = result_df.select_dtypes(include=['object', 'category']).columns.tolist()
    
    if not categorical_columns:
        print("No categorical columns found to encode.")
        return result_df, None
        
    # Initialize the encoder
    encoder = OneHotEncoder(sparse_output=False, handle_unknown=handle_unknown, drop='if_binary')
    
    # Fit and transform the categorical columns
    encoded_array = encoder.fit_transform(result_df[categorical_columns])
    
    # Get feature names
    feature_names = encoder.get_feature_names_out(categorical_columns)
    
    # Create a DataFrame with the encoded features
    encoded_df = pd.DataFrame(encoded_array, columns=feature_names, index=result_df.index)
    
    # Combine with the original DataFrame
    if drop_original:
        # Drop the original categorical columns
        result_df = result_df.drop(columns=categorical_columns)
    
    # Concatenate the encoded columns with the original DataFrame
    result_df = pd.concat([result_df, encoded_df], axis=1)
    
    print(f"One-hot encoded {len(categorical_columns)} categorical columns into {len(feature_names)} binary features.")
    
    return result_df, encoder

In [9]:
# Apply to your training data
train_data_oh, encoder = one_hot_encode_categories(train_data, handle_unknown='ignore')

One-hot encoded 4 categorical columns into 17 binary features.


In [17]:
train_data_oh.head()

,customer_ID,S_2,D_39,B_1,B_2,R_1,D_41,B_3,B_4,D_45,B_5,R_2,D_47,B_6,B_7,D_51,B_9,R_3,B_10,S_5,B_11,S_6,D_54,R_4,B_12,S_8,B_13,R_5,D_58,B_14,D_60,S_11,D_65,B_16,B_18,B_19,B_20,S_12,R_6,S_13,B_21,B_22,D_71,S_15,B_23,P_4,D_75,B_24,R_7,B_26,R_8,S_16,R_10,R_11,B_27,S_17,R_12,B_28,R_13,R_14,R_15,R_16,S_18,D_86,R_17,R_18,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,R_22,R_23,D_92,D_93,D_94,R_24,R_25,D_96,D_102,B_36,D_127,D_133,R_28,D_140,D_144,target,end_of_month,D_63_CL,D_63_CO,D_63_CR,D_63_XL,D_63_XM,D_63_XZ,B_30_0.0,B_30_1.0,B_30_2.0,B_38_1.0,B_38_2.0,B_38_3.0,B_38_4.0,B_38_5.0,B_38_6.0,B_38_7.0,D_126_1.0
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2018-02-21,0.002425,0.019837,1.008307,0.000607,0.006174,0.007454,0.080204,0.740062,0.132626,0.001436,0.538275,0.150155,0.060502,0.670537,0.001064,0.009852,0.325464,0.050146,0.007260,0.006510,1.001046,0.004440,0.111060,0.755829,0.092181,0.002498,0.005720,0.029953,0.141639,0.561116,0.009596,0.005605,1.005261,0.007416,0.009710,0.187413,0.004807,0.846702,0.003347,0.006082,0.312937,0.200894,0.044315,0.003468,0.009681,0.008714,0.003488,0.000013,0.008551,0.005027,0.004827,0.008204,0.002737,0.007090,1.001785,0.098844,0.009521,0.004815,0.009732,0.008460,0.007482,0.003417,0.004134,0.007100,1,0.000869,0.002693,0.008749,0.004063,0.005139,0.007752,1.004296,0.000684,0.000934,0.004344,0.004264,0.007412,0.004308,0.006280,0.005712,0.922532,0.009383,1.007527,0.000122,0.007725,0.001976,0.006346,0,2018-02-28,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,2018-02-06,0.536290,0.034558,1.005419,0.007248,0.006274,0.008733,0.020876,0.263736,0.010155,0.003805,0.398607,0.211673,0.033179,0.340006,0.011734,0.109444,0.301616,0.006361,0.015018,0.005666,1.005178,0.000279,0.022597,0.319952,0.008499,0.003330,0.008624,0.023292,0.524772,0.444329,0.000660,0.003267,1.001247,0.002385,0.000350,0.188421,0.002118,0.288901,0.008382,0.009769,0.010515,0.406084,0.014946,0.005555,0.009073,0.009167,0.009282,0.007601,0.006927,0.000067,0.004670,0.005223,0.000015,0.003619,1.002889,0.014986,0.003408,0.009129,0.003361,0.009003,0.005150,0.002534,0.002449,0.007003,1,0.009282,0.000871,0.005917,0.009783,0.001053,0.006891,1.000591,0.005175,0.001321,0.008394,0.005752,0.009342,0.006949,0.005339,0.002806,0.001432,0.004645,0.003569,0.009592,0.006052,0.004627,0.002528,0,2018-02-28,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,2018-02-09,0.001526,0.008318,0.818953,0.006756,0.009511,0.004373,0.057312,0.245516,0.000215,0.009075,0.339566,0.178068,0.042980,0.337689,0.000422,0.008294,0.302024,0.004721,0.007528,1.006839,1.006134,0.001380,0.006100,0.009502,0.002143,0.008246,0.009205,0.002675,0.007859,0.288868,0.005108,0.005106,1.007293,0.001920,0.009832,0.187889,0.000800,0.000768,0.006931,0.001579,0.013982,0.508765,0.028005,0.007414,0.009046,0.003384,0.005028,0.003570,0.002032,0.001470,0.006335,0.009125,0.005929,0.006845,1.007555,0.050830,0.000085,0.005226,0.009405,0.000760,0.001418,0.007448,0.000301,0.007136,1,0.008780,0.006727,0.004732,0.003573,0.007283,0.006645,1.004516,0.006888,0.001481,0.005212,0.005087,0.001395,0.002500,0.006046,0.009363,0.004414,0.006159,0.002729,0.001980,0.005101,0.009543,0.007325,0,2018-02-28,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,2018-02-03,0.008380,0.014589,1.009999,0.003768,0.008239,0.004804,0.008486,0.074935,0.115955,0.009204,0.410036,0.195544,0.012364,0.008012,0.034357,0.001734,0.401879,0.030321,0.004926,0.009158,1.004057,0.003462,0.075619,0.174018,0.060876,0.003396,0.006757,0.016034,0.885493,0.367468,0.001220,0.169134,1.006742,0.002138,0.001995,0.350610,0.004044,0.423225,0.009143,0.005501,0.012520,0.501547,0.003451,0.008617,0.002992,0.005909,0.002774,0.003175,0.004093,0.003130,0.004100,0.003846,0.000866,0.004267,1.003564,0.017212,0.001522,0.006544,0.009200,0.003933,0.008703,0.002066,0.008195,0.004

In [12]:
encoder.feature_names_in_

array(['D_63', 'B_30', 'B_38', 'D_126'], dtype=object)

In [15]:
encoder.transform(test_data[encoder.feature_names_in_])

array([[0., 0., 1., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.],
       ...,
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.]])

In [19]:
test_data_oh = pd.concat([test_data.drop(columns=encoder.feature_names_in_),pd.DataFrame(encoder.transform(test_data[encoder.feature_names_in_]), columns=encoder.get_feature_names_out(),index=test_data.index)], axis=1)

In [21]:
test_data_oh.shape

(391349, 108)

In [50]:
from sklearn.model_selection import train_test_split
X_train, X_val = train_test_split(train_data_oh, test_size=0.3, random_state=0, stratify = train_data_oh['target'])

In [51]:
y_train = X_train['target']
y_val = X_val['target']

In [52]:
X_train.shape, X_val.shape

((270223, 108), (115811, 108))

In [29]:
X_train[['customer_ID', 'target']].to_parquet('../../data/initial_modelling/y_train.parquet', index = False)
X_val[['customer_ID', 'target']].to_parquet('../../data/initial_modelling/y_val.parquet', index = False)

In [53]:
drop_cols = ['customer_ID', 'target', 'S_2', 'end_of_month']

In [54]:
X_train = X_train.drop(columns = drop_cols)
X_val = X_val.drop(columns = drop_cols)

In [55]:
X_val.head(10)

,D_39,B_1,B_2,R_1,D_41,B_3,B_4,D_45,B_5,R_2,D_47,B_6,B_7,D_51,B_9,R_3,B_10,S_5,B_11,S_6,D_54,R_4,B_12,S_8,B_13,R_5,D_58,B_14,D_60,S_11,D_65,B_16,B_18,B_19,B_20,S_12,R_6,S_13,B_21,B_22,D_71,S_15,B_23,P_4,D_75,B_24,R_7,B_26,R_8,S_16,R_10,R_11,B_27,S_17,R_12,B_28,R_13,R_14,R_15,R_16,S_18,D_86,R_17,R_18,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,R_22,R_23,D_92,D_93,D_94,R_24,R_25,D_96,D_102,B_36,D_127,D_133,R_28,D_140,D_144,D_63_CL,D_63_CO,D_63_CR,D_63_XL,D_63_XM,D_63_XZ,B_30_0.0,B_30_1.0,B_30_2.0,B_38_1.0,B_38_2.0,B_38_3.0,B_38_4.0,B_38_5.0,B_38_6.0,B_38_7.0,D_126_1.0
88325,0.120957,0.035195,1.004083,0.004925,0.001032,0.008988,0.004605,0.203044,0.075450,0.006382,0.735497,0.298437,0.027460,0.337887,0.051187,0.000003,0.296798,0.073150,0.017205,0.004084,1.004965,0.000043,0.032905,0.665858,0.038356,0.000374,0.006220,0.017933,1.005045,0.243441,0.002926,0.092435,1.007904,0.003588,0.009979,0.329074,0.005426,0.430447,0.005244,0.003521,0.008785,0.206735,0.015719,0.003484,0.003096,0.003422,0.006192,0.002962,0.002545,0.009316,0.009305,0.009885,0.006346,0.009987,1.003134,0.009227,0.008306,0.004441,0.008891,0.003154,0.002763,0.005386,0.008888,0.001875,1,0.004231,0.009065,0.001634,0.004603,0.006467,0.004760,1.003279,0.009976,0.000156,0.002362,0.009334,0.008191,0.000628,0.008596,0.008027,0.005424,0.005984,0.009219,0.001255,0.004179,0.002441,0.007002,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
193017,0.001720,0.001524,0.813657,0.000218,0.001056,0.009691,0.014720,0.056135,0.000175,0.007640,0.985860,0.181085,0.031458,0.002181,0.009560,0.003043,0.296927,0.006165,0.003032,1.005942,1.002414,0.001315,0.013563,0.007538,0.005112,0.002641,0.007170,0.003924,0.005425,0.281230,0.001440,0.000290,1.008139,0.002709,0.009276,0.189697,0.005730,0.007814,0.002408,0.000216,0.010108,0.503985,0.016175,0.008739,0.007404,0.001946,0.008612,0.009489,0.003646,0.002551,0.009739,0.000797,0.009569,0.007202,1.000197,0.014193,0.003337,0.004228,0.009351,0.006876,0.004256,0.004741,0.003319,0.003399,1,0.005198,0.001395,0.000272,0.005588,0.008388,0.001862,1.004562,0.003931,0.007479,0.002614,0.003106,0.007311,0.009697,0.003515,0.001204,0.175911,0.004604,0.006039,0.002105,0.003323,0.004103,0.009339,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
40534,0.471850,0.018508,0.813154,0.007822,0.008074,0.064667,0.233385,0.050480,0.001866,0.008061,0.057846,0.038433,0.271255,0.007484,0.012184,0.102959,0.055980,0.025713,0.010016,0.002763,1.001114,0.008404,0.041913,0.104340,0.090945,0.003674,0.211854,0.016622,0.027233,0.365584,0.006827,0.251002,0.584839,0.258146,0.000790,0.224926,0.007859,0.005775,0.012626,0.008864,0.011425,0.707718,0.257138,0.008765,0.202443,0.002255,0.002568,0.000449,0.005869,0.035506,0.002641,0.009004,0.000636,0.003298,1.007228,0.170887,0.008695,0.000537,0.005625,0.007995,0.003497,0.006232,0.003083,0.007430,1,0.001157,0.000848,0.006349,0.001337,0.001023,0.007651,1.004773,0.006614,0.006662,0.000264,0.004536,0.001058,0.005747,0.001888,0.002615,0.001948,0.004887,0.006033,0.008546,0.006798,0.007247,0.002695,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
176556,0.005953,0.007611,0.819612,0.008702,0.005017,0.002600,0.157042,0.278636,0.000282,0.006101,0.351115,0.033643,0.235353,0.334022,0.005273,0.004867,0.033581,0.002144,0.004070,1.000365,1.003282,0.004653,0.006817,0.007157,0.006367,0.007687,0.635153,0.004225,0.002586,0.287302,0.001020,0.000535,0.532575,0.000974,0.000730,0.187517,0.009375,0.004795,0.001050,0.000376,0.015446,0.505249,0.217490,0.005107,0.402494,0.009637,0.005589,0.007935,0.001054,0.002765,0.008124,0.002988,0.003555,0.001810,1.008538,0.189585,0.008112,0.001808,0.004845,0.006352,0.005411,0.003592,0.005444,0.002983,1,0.006697,0.007942,0.008503,0.008916,0.008992,0.008573,1.003086,0.001257,0.001356,0.009462,0.006382,0.007623,0.006569,0.002384,0.003940,0.439094,0.008016,0.006517,0.006614,0.009997,0.005797,0.672997,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
25107,0.914097,0.372201,0.062

In [56]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

In [57]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns = X_train.columns)
X_val_scaled = pd.DataFrame(X_val_scaled, columns = X_train.columns)

In [58]:
X_train_scaled.head(10)

,D_39,B_1,B_2,R_1,D_41,B_3,B_4,D_45,B_5,R_2,D_47,B_6,B_7,D_51,B_9,R_3,B_10,S_5,B_11,S_6,D_54,R_4,B_12,S_8,B_13,R_5,D_58,B_14,D_60,S_11,D_65,B_16,B_18,B_19,B_20,S_12,R_6,S_13,B_21,B_22,D_71,S_15,B_23,P_4,D_75,B_24,R_7,B_26,R_8,S_16,R_10,R_11,B_27,S_17,R_12,B_28,R_13,R_14,R_15,R_16,S_18,D_86,R_17,R_18,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,R_22,R_23,D_92,D_93,D_94,R_24,R_25,D_96,D_102,B_36,D_127,D_133,R_28,D_140,D_144,D_63_CL,D_63_CO,D_63_CR,D_63_XL,D_63_XM,D_63_XZ,B_30_0.0,B_30_1.0,B_30_2.0,B_38_1.0,B_38_2.0,B_38_3.0,B_38_4.0,B_38_5.0,B_38_6.0,B_38_7.0,D_126_1.0
0,-0.092340,-0.179184,0.005511,-0.358213,-0.830086,-0.014621,-0.248679,0.398645,-0.298152,-0.895949,1.743955,0.778834,-0.168891,1.003377,-0.055942,-0.070220,0.701141,-0.075761,-0.117106,154.671111,-0.731415,-0.524039,-0.099336,-0.637713,-0.258112,0.624936,-0.264184,-0.240600,-0.346620,-0.189953,-0.194801,-0.093350,0.453367,0.006936,0.000174,0.348935,-0.363672,-0.018399,0.010298,0.370727,-0.200185,0.361136,-0.176685,-0.343606,-0.249899,0.596609,-0.839411,0.848318,0.415128,-0.495343,0.555486,-0.250483,-0.480877,0.118248,-0.505002,-0.293364,-0.821713,0.768591,-0.630215,-0.541403,-0.847114,0.235637,0.548836,-0.739062,0.0,0.478225,-0.315801,-0.617955,0.876819,-0.494403,0.880331,0.001481,0.140212,-0.403684,0.145079,-0.984856,-0.387010,0.281075,0.634471,-0.581540,2.060366,-0.615877,0.029447,-0.952089,-0.240178,0.522962,-0.580366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.612272,-0.005333,0.208336,-0.718587,0.444595,-0.022155,-0.014321,-0.389799,-0.104260,-0.598490,-0.180305,1.326962,-0.138426,1.004300,0.134976,-0.078017,0.685506,5.531883,-0.012388,-0.895955,-0.794075,0.055619,2.595322,0.358920,1.595477,0.637655,-0.267043,0.523625,0.584228,-0.564003,-0.085233,-0.001965,0.446397,-0.022646,-0.013002,17.082814,-0.381787,0.990588,0.946708,0.087701,-0.424692,-0.340223,-0.139413,0.109099,-0.252935,0.226885,0.213508,-0.388033,-0.353623,-0.677864,-0.709799,-0.217832,-0.478303,0.531035,0.958592,-0.064985,0.031016,0.751804,0.936601,-0.361093,-0.752776,-0.319844,0.505865,0.389486,0.0,-0.224250,-0.968776,-0.169272,-0.659069,0.705056,0.298055,-0.000253,0.605604,0.225894,181.884884,-0.649920,0.374676,0.343857,-0.624992,0.642741,-0.014972,0.875184,-0.618548,-0.575995,-0.909723,-0.025012,0.717438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.137318,0.053736,0.203783,-0.938258,0.201177,-0.012908,-0.360850,-0.204697,0.699555,-0.761914,-0.750451,1.887921,-0.132840,0.992581,0.004030,-0.067945,0.873174,0.308653,0.008755,-0.329931,-0.850768,-0.141533,-0.144128,-0.002332,0.023663,0.568464,-0.283372,-0.122189,1.120974,-0.372662,-0.996431,-0.091692,0.443375,-0.035860,-0.013450,8.788065,-0.709632,0.648917,-0.557632,-0.449543,0.052178,-0.004003,-0.168891,0.587226,-0.265388,-0.610351,0.412703,0.783056,-0.140330,0.751150,-0.962227,-0.416723,0.943866,-0.547002,0.668608,-0.384925,-0.442428,-0.574936,0.775303,-0.974987,0.934835,-0.679840,-0.273421,-0.977952,0.0,0.502210,0.752285,-0.952262,0.863390,0.261182,0.046568,-0.000142,-0.565200,0.356735,0.058313,0.817910,0.196444,0.439304,0.767467,-0.714939,-0.000380,0.305645,0.510251,-0.782966,0.922314,-0.720903,-0.311874,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.0
3,-0.096258,-0.207411,0.002801,0.212193,-0.439213,-0.041781,-0.348323,0.389570,-0.230332,0.683254,0.030338,0.599590,-0.224658,-0.016679,-0.045391,1.910875,0.711462,-0.136672,-0.154588,0.520753,0.533645,0.174473,0.101968,-0.645727,0.266643,0.311671,-0.278495,-0.234381,-0.346581,-0.181455,0.237741,0.089456,0.443171,-0.016716,-0.013732,-0.087156,-0.910955,-0.012375,-0.906951,0.013277,-0.193512,0.334293,-0.192001,0.382583,-0.258409,0.107352,-0.240672,-0.662332,0.037259,-0.155389,0.636830,93.643924,-0.937919,0.162300,-0.102432,-0.381439,-0.574382,0.888804,0.845828,0.649736,0.336668,-0.791629,-0.706961,0.439333,0.0,-0.809960,-0.391681,-0.021612,0.789243,-0.517037,-0.525999,0.003355,0.047938,0.285281,0.826397,0.879444,-0.769798,-0.735

In [59]:
X_val_scaled.head(10)

,D_39,B_1,B_2,R_1,D_41,B_3,B_4,D_45,B_5,R_2,D_47,B_6,B_7,D_51,B_9,R_3,B_10,S_5,B_11,S_6,D_54,R_4,B_12,S_8,B_13,R_5,D_58,B_14,D_60,S_11,D_65,B_16,B_18,B_19,B_20,S_12,R_6,S_13,B_21,B_22,D_71,S_15,B_23,P_4,D_75,B_24,R_7,B_26,R_8,S_16,R_10,R_11,B_27,S_17,R_12,B_28,R_13,R_14,R_15,R_16,S_18,D_86,R_17,R_18,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,R_22,R_23,D_92,D_93,D_94,R_24,R_25,D_96,D_102,B_36,D_127,D_133,R_28,D_140,D_144,D_63_CL,D_63_CO,D_63_CR,D_63_XL,D_63_XM,D_63_XZ,B_30_0.0,B_30_1.0,B_30_2.0,B_38_1.0,B_38_2.0,B_38_3.0,B_38_4.0,B_38_5.0,B_38_6.0,B_38_7.0,D_126_1.0
0,0.309842,0.035923,0.205886,-0.153049,-0.825474,-0.004669,-0.341864,0.009004,1.201696,0.197299,1.062378,1.260657,-0.176534,0.992158,0.091281,-0.081772,0.697140,0.965958,-0.013762,-0.367047,0.020457,-0.991550,0.218162,0.709964,0.132685,-0.924790,-0.269327,-0.120081,1.211725,-0.396423,-0.456662,-0.000973,0.451345,-0.022345,0.002932,14.238489,0.027615,0.994859,0.029635,-0.449135,-0.165227,-0.658765,-0.170371,-0.319911,-0.270158,-0.342274,0.187452,-0.450444,-0.510801,0.827486,0.754647,0.858613,0.266187,0.846251,-0.325509,-0.403889,0.641107,-0.132590,0.748681,-0.412773,-0.465632,0.042278,0.762465,-0.625089,0.0,-0.153403,0.772195,-0.684007,-0.088177,0.265657,-0.061047,0.001756,0.987343,-0.966865,-0.565628,0.847238,0.608355,-0.879462,0.710649,0.554826,-0.010433,0.185048,0.633481,-0.769676,-0.165345,-0.520131,0.270910,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.097772,-0.210495,-0.000336,-0.963083,-0.821358,-0.001417,-0.299217,-0.450626,-0.322064,0.433237,1.839608,0.586058,-0.161671,-0.016070,-0.028727,-0.052426,0.697622,-0.093540,-0.136793,155.035181,-0.472506,-0.747581,-0.100020,-0.642644,-0.275228,-0.491564,-0.266868,-0.252959,-0.340394,-0.208973,-0.733784,-0.101750,0.451638,-0.027817,0.001730,-0.090025,0.085247,-0.003527,-0.525894,-0.970069,-0.113388,0.340926,-0.168606,0.704929,-0.253854,-0.626758,0.652034,0.764908,-0.297974,-0.500965,0.836554,-0.851166,0.910891,0.331274,-0.892813,-0.377520,-0.340702,-0.174199,0.839072,0.283127,-0.174887,-0.082612,-0.342411,-0.319930,0.0,0.040040,-0.724616,-0.949420,0.107323,0.642081,-0.631277,0.003040,-0.215389,0.497258,-0.519663,-0.384583,0.435634,0.908686,-0.302536,-0.765975,0.493508,-0.087887,0.070907,-0.614378,-0.336052,-0.193821,0.695384,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.509376,-0.086202,-0.000881,0.345405,0.366370,0.252745,0.622717,-0.468319,-0.287838,0.512103,-1.041334,-0.233976,0.729844,-0.000143,-0.021164,0.912152,-0.198478,0.215638,-0.076166,-0.571977,-0.723652,0.611633,0.366346,-0.443752,0.777989,-0.294187,0.263468,-0.132519,-0.306533,0.209459,0.270666,0.172449,-0.074998,1.562491,-0.012765,3.531686,0.488965,-0.008342,1.475907,0.393145,-0.061738,1.026103,0.763850,0.710068,0.484308,-0.567219,-0.508303,-0.918516,0.131645,5.970287,-0.503524,0.692968,-0.876018,-0.390722,0.465102,0.454449,0.717870,-0.896152,0.106414,0.492312,-0.322767,0.206047,-0.389183,0.487056,0.0,-0.768193,-0.831357,0.234777,-0.735922,-0.801319,0.507541,0.003252,0.318480,0.334011,-0.948387,-0.101696,-0.791603,0.129738,-0.627074,-0.492915,-0.020707,-0.031788,0.069741,0.562563,0.357523,0.423691,-0.511167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.083300,-0.165946,0.006113,0.496752,-0.150923,-0.034202,0.300841,0.245507,-0.319893,0.144577,-0.130903,-0.261510,0.596365,0.980551,-0.041087,-0.034821,-0.281784,-0.157153,-0.127779,154.170097,-0.304752,-0.107481,-0.210999,-0.643428,-0.259835,0.472818,1.360235,-0.250102,-0.344802,-0.178855,-0.811915,-0.101481,-0.140020,-0.038617,-0.012868,-0.314186,0.776600,-0.010659,-0.791888,-0.944863,0.095837,0.345175,0.610422,-0.003408,1.241443,0.855123,0.071584,0.475485,-0.798848,-0.458903,0.531679,-0.439052,-0.292009,-0.665818,0.718247,0.553724,0.602700,-0.647566,-0.046957,0.185244,0.050137,-0.305075,0.079207,-0.403216,0.0,0.339731,0.553075,0.654526,0.767420,0.760343,0.689071,0.001564,-0.747500,-0.726912,0.730135,0.263408,0.496817,0.291887,

In [60]:
from xgboost import XGBClassifier

In [61]:
model = XGBClassifier(random_state=0)

In [62]:
model.fit(X_train_scaled, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=0, ...)

In [63]:
y_pred = model.predict(X_val_scaled)

In [66]:
from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.93      0.93     88967
           1       0.78      0.76      0.77     26844

    accuracy                           0.89    115811
   macro avg       0.85      0.85      0.85    115811
weighted avg       0.89      0.89      0.89    115811



In [67]:
# Test set results

In [68]:
test_data_oh.drop(columns = drop_cols, inplace = True)

In [69]:
test_X_scaled = scaler.transform(test_data_oh)
test_X_scaled = pd.DataFrame(test_X_scaled, columns = test_data_oh.columns)

In [71]:
y_test = test_data['target']

In [72]:
y_pred_test = model.predict(test_X_scaled)

In [73]:
print(classification_report(y_test, y_pred_test, digits = 4))

              precision    recall  f1-score   support

           0     0.9346    0.9367    0.9356    299141
           1     0.7931    0.7874    0.7902     92208

    accuracy                         0.9015    391349
   macro avg     0.8638    0.8620    0.8629    391349
weighted avg     0.9013    0.9015    0.9014    391349

